# Implementing First Order Advection

We now have all the pieces needed to implement first-order accurate advection.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

For convenience, the grid module we already saw can by imported

In [3]:
import fv_grid

Now we'll write our driver

In [ ]:
def advection(nx, u, C, num_periods=1, init_cond=None):